In [17]:
!pip install trl  bitsandbytes flwr flwr_datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.0/87.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 80.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.9.0
    Uninstalling fsspec-2025.9.0:
      Successfully uninstalled fsspec-2025.9.0
  Attempting uninstall: dill
    Found existing installation: dill 0.4.0
    Uninstalling dill-0.4.0:
      Successfully uninstalled dill-0.4.0
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.2
    Uninstalling matplotlib-3.7.2:
      Successfully uninstalled matplotlib-3.7.2
  Attempting uninstall: seaborn
    Fo

In [ ]:
from huggingface_hub import login
login(token="YOUR-API-KEY")
import wandb

wandb.login(key="your-api-key")

/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
/usr/local/lib/python3.11/dist-packages/google/colab/_import_hooks/_pydrive.py:21: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp  # pylint: disable=deprecated-module
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: tuhanasinan (tuhanasinan-t-c-i-stanbul-sabahattin-zaim-niversitesi) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [7]:
from omegaconf import OmegaConf, DictConfig

cfg_dict = {
    'dataset': {
        'name': 'medalpaca/medical_meadow_medical_flashcards'
    },
    'model': {
        'name': "meta-llama/Llama-3.2-1B-Instruct",
        'quantization': 4,
        'gradient_checkpointing': True,
        'use_fast_tokenizer': True,
        'lora': {
            'peft_lora_r': 16,
            'peft_lora_alpha': 64,
            'target_modules': None}},
    'train': {
        'num_rounds': '${flower.num_rounds}',
        'save_every_round': 5,
        'learning_rate_max': 5e-05,
        'learning_rate_min': 1e-06,
        'seq_length': 1024,
        'padding_side': 'left',
        'evaluate_split': True,
        'training_arguments': {
            'output_dir': None,
            'learning_rate': None,
            'per_device_train_batch_size': 2,
            'gradient_accumulation_steps': 1,
            'logging_steps': 5,
            'max_steps': 250,
            # 'num_train_epochs': 1,
            'report_to': 'wandb',
            'run_name': "llama32_1b_fed_sft_take2",
            'save_steps': 1000,
            'save_total_limit': 10,
            'gradient_checkpointing': '${model.gradient_checkpointing}',
            'lr_scheduler_type': 'constant'}},
    'flower': {
        'num_clients': 20,
        'num_rounds': 2,
        'fraction_fit': 0.2,
        'client_resources': {
            'num_cpus': 2,
            'num_gpus': 1.0
        },
        'dp': {
            'noise_mult': 0.02,
            'clip_norm': 0.5
        }
    }
}

cfg = OmegaConf.create(cfg_dict)

In [8]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments  # SFTConfig için eklendi
)
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
)
from peft.utils import prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig  # <-- Burası düzeltildi

def get_model(model_cfg: DictConfig):
    """Load model with appropiate quantization config and
    other optimizations. Notice here that we are returning the
    LoRA model, not the full model."""

    use_cuda = torch.cuda.is_available()
    quantization_config = None
    model_name = model_cfg.name
    if use_cuda:
        if model_cfg.quantization == 4:
            quantization_config = BitsAndBytesConfig(load_in_4bit=True)
        elif model_cfg.quantization == 8:
            quantization_config = BitsAndBytesConfig(load_in_8bit=True)
        else:
            raise ValueError(
                f"Use 4-bit or 8-bit quantization. You passed: {model_cfg.quantization}/"
            )

        model_name = model_cfg.name

    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=quantization_config,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
    )

    if use_cuda:
        model = prepare_model_for_kbit_training(
            model, use_gradient_checkpointing=model_cfg.gradient_checkpointing
        )

    target_modules = model_cfg.lora.target_modules
    if target_modules:
        target_modules = list(target_modules)

    peft_config = LoraConfig(
        r=model_cfg.lora.peft_lora_r,
        lora_alpha=model_cfg.lora.peft_lora_alpha,
        lora_dropout=0.075,
        task_type="CAUSAL_LM",
        target_modules=target_modules,
    )

    peft_model = get_peft_model(model, peft_config)
    if not (use_cuda):
        peft_model.enable_input_require_grads()

    if model_cfg.gradient_checkpointing:
        model.config.use_cache = False

    return peft_model

def compute_communication_costs(config, comm_bw_mbps: float = 20):
      """
      We use Flower's helper function to compute communication costs of federated finetuning.
      Get ready to see the savings!
      """
      model = get_model(config.model)

      trainable, all_parameters = model.get_nb_trainable_parameters()

      total_size = 4*all_parameters/(1024**2)
      trainable_size = 4*trainable/(1024**2)

      upload_time_total = total_size/(comm_bw_mbps/8) # Convert Mbps to MBps by dividing by 8
      upload_time_finetune = trainable_size/(comm_bw_mbps/8)

      print(f"Full model:\n\t{all_parameters/1e6:.3f} M parameters\n\t{total_size:.2f} MB --> upload in {upload_time_total:.2f}s @ {comm_bw_mbps}Mbps")
      print(f"Finetuned model:\n\t{trainable/1e6:.3f} M parameters\n\t{trainable_size:.2f} MB --> upload in {upload_time_finetune:.2f}s @ {comm_bw_mbps}Mbps")
      # print(f"In a {comm_bw_mbps} Mbps channel --> {}")

      num_rounds = config.flower.num_rounds
      num_clients_per_round = int(config.flower.num_clients * config.flower.fraction_fit)
      print(f"Federated Learning setting: "
            f"\n\tNumber of rounds: {num_rounds}"
            f"\n\tNumber of clients per round: {num_clients_per_round}")

      print(f"-----------------------------------------------")
      print(f"Total Communication costs (Full model): {2*num_rounds*num_clients_per_round*total_size/1024:.1f} GB")
      print(f"Total Communication costs (Finetuning): {2*num_rounds*num_clients_per_round*trainable_size} MB")
      print(f"Communication savings: {all_parameters/trainable:.1f}x")

compute_communication_costs(cfg, comm_bw_mbps=20)

`torch_dtype` is deprecated! Use `dtype` instead!


Full model:
	1237.518 M parameters
	4720.76 MB --> upload in 1888.30s @ 20Mbps
Finetuned model:
	1.704 M parameters
	6.50 MB --> upload in 2.60s @ 20Mbps
Federated Learning setting: 
	Number of rounds: 2
	Number of clients per round: 4
-----------------------------------------------
Total Communication costs (Full model): 73.8 GB
Total Communication costs (Finetuning): 104.0 MB
Communication savings: 726.3x


In [45]:
from transformers import AutoTokenizer # Gerekli import'u en üste taşıdım
from omegaconf import DictConfig # cfg için gerekli

def formatting_prompts_func(example):
    """
    Constructing a standard Alpaca prompt for a SINGLE example.
    'example' is a dictionary (a single row), not a batch.
    """
    mssg = "Below is an instruction that describes a task. Write a response that appropriately completes the request."
    text = f"{mssg}\n### Instruction:\n{example['instruction']}\n### Response: {example['response']}"
    return text

def get_tokenizer_and_data_collator_and_prompt_formatting(
    model_name: str, use_fast: bool, padding_side: str
):
    """Loads tokenizer, sets padding, returns None for collator."""
    tokenizer = AutoTokenizer.from_pretrained(
        model_name, use_fast=use_fast, padding_side=padding_side
    )
    tokenizer.pad_token = (
        tokenizer.bos_token if padding_side == "left" else tokenizer.eos_token
    )
    # TRL 0.24.0 için DataCollator'ı None olarak ayarlamak doğrudur.
    data_collator = None
    # Düzeltilmiş (doğru) fonksiyonu döndürür
    return tokenizer, data_collator, formatting_prompts_func

# Fonksiyonu SADECE BİR KERE çağırın
tokenizer, data_collator, formatting_prompts_func = get_tokenizer_and_data_collator_and_prompt_formatting(
    cfg.model.name,
    cfg.model.use_fast_tokenizer,
    cfg.train.padding_side,
)

print("✅ Tokenizer, data_collator (None), ve formatting_prompts_func tanımlandı (Doğru Versiyon).")

✅ Tokenizer, data_collator (None), ve formatting_prompts_func tanımlandı (Doğru Versiyon).


In [10]:
import math

def cosine_annealing(
    current_round: int,
    total_round: int,
    lrate_max: float = 0.001,
    lrate_min: float = 0.0,
) -> float:
    """Implement cosine annealing learning rate schedule. Strictly speaking this
    is not necessary."""

    cos_inner = math.pi * current_round / total_round
    return lrate_min + 0.5 * (lrate_max - lrate_min) * (1 + math.cos(cos_inner))

In [51]:
import torch
from flwr.common import Context
from flwr.common.typing import NDArrays, Scalar
from flwr.client import NumPyClient
from typing import Dict, Tuple, Callable
from collections import OrderedDict
from trl import SFTConfig, SFTTrainer
from omegaconf import DictConfig, OmegaConf # Gerekli import eklendi
from peft import get_peft_model_state_dict, set_peft_model_state_dict
# Not: get_model, cosine_annealing ve set_parameters fonksiyonlarınızın
# bu kod bloğunun dışında bir yerde zaten tanımlı olduğunu varsayıyorum.

# ---------------------------------------------------------------------------
#                           DÜZELTİLMİŞ KOD
# ---------------------------------------------------------------------------

class FlowerClient(NumPyClient):
    def __init__(
        self,
        model_cfg: DictConfig,
        train_cfg: DictConfig,
        trainset,
        tokenizer,
        formatting_prompts_func,
        data_collator, 
        save_path,
    ):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.train_cfg = train_cfg
        
        # --- DÜZELTME 1: SFTConfig 'completion_only_loss' ve 'dataset_text_field' için güncellendi ---
        sft_config_dict = OmegaConf.to_container(train_cfg.training_arguments, resolve=True)
        
        self.training_arguments = SFTConfig(
            **sft_config_dict,
            completion_only_loss=True,  # Sadece yanıtı eğit
            dataset_text_field="text"   # Formatlanmış verinin sütun adı
        )
        # --- DÜZELTME 1 SONU ---

        self.tokenizer = tokenizer
        self.formatting_prompts_func = formatting_prompts_func
        self.data_collator = data_collator
        self.save_path = save_path
        self.model = get_model(model_cfg)
        self.trainset = trainset

    def get_parameters(self, config: Dict[str, Scalar]) -> NDArrays:
        """Return the parameters of the current net."""
        state_dict = get_peft_model_state_dict(self.model)
        return [val.cpu().numpy() for _, val in state_dict.items()]

    def fit(
        self, parameters: NDArrays, config: Dict[str, Scalar]
    ) -> Tuple[NDArrays, int, Dict]:
        """Implement distributed fit function for a given client."""
        set_parameters(self.model, parameters)

        new_lr = cosine_annealing(
            int(config["current_round"]),
            self.train_cfg.num_rounds,
            self.train_cfg.learning_rate_max,
            self.train_cfg.learning_rate_min,
        )

        self.training_arguments.learning_rate = new_lr
        self.training_arguments.output_dir = self.save_path

        evalset = None
        if self.train_cfg.evaluate_split:
            train_test = self.trainset.train_test_split(test_size=0.1, seed=1234)
            trainset = train_test['train']
            evalset = train_test['test']
        else:
            trainset = self.trainset

        # --- DÜZELTME 2: Formatlama SFTTrainer'dan ÖNCE yapılır ---
        
        # Veri setini formatlamak için (doğru 'formatting_prompts_func' kullanılır)
        def format_dataset(example):
            return {"text": self.formatting_prompts_func(example)}

        # .map() ile formatlamayı uygula
        trainset = trainset.map(format_dataset, batched=False)
        if evalset:
            evalset = evalset.map(format_dataset, batched=False)
        # --- DÜZELTME 2.A SONU ---

        trainer = SFTTrainer(
            model=self.model,
            processing_class=self.tokenizer, # 'processing_class' yerine 'tokenizer' kullanılır
            args=self.training_arguments, 
            train_dataset=trainset,       
            eval_dataset=evalset,
            # 'formatting_func' kaldırıldı (çünkü önceden mapledik)
            data_collator=self.data_collator,
        )
        # --- DÜZELTME 2.B SONU ---

        metrics = {}
        if self.train_cfg.evaluate_split:
            eval_res = trainer.evaluate()
            metrics['eval_loss'] = eval_res['eval_loss']
            print(eval_res)

        results = trainer.train()
        metrics = {**metrics, "train_loss": results.training_loss}

        return (
            self.get_parameters({}),
            len(self.trainset),
            metrics,
        )

def gen_client_fn(
    fds,
    tokenizer,
    formatting_prompts_func,
    data_collator,
    model_cfg: DictConfig,
    train_cfg: DictConfig,
    save_path: str,
) -> Callable[[str], FlowerClient]:
    """Generate the client function that creates the Flower Clients."""

    def client_fn(context: Context) -> FlowerClient:
        """Create a Flower client representing a single organization."""

        partition_id = int(context.node_config["partition-id"])
        client_trainset = fds.load_partition(partition_id, "train")

        # --- DÜZELTME BURADA ---
        # Veri setinde zaten 'instruction' var. 'input'u silelim.
        # 'output'u 'response' yapalım.
        client_trainset = client_trainset.remove_columns(["input"])
        client_trainset = client_trainset.rename_column("output", "response")
        # --- DÜZELTME SONU ---
        
        # Artık sütunlar: ['output', 'instruction'] -> ['response', 'instruction']
        # formatting_prompts_func bunları kullanabilir.
        return FlowerClient(
            model_cfg,
            train_cfg,
            client_trainset,
            tokenizer,
            formatting_prompts_func,
            data_collator,
            save_path,
        ).to_client()

    return client_fn

In [52]:
from flwr.client.mod import fixedclipping_mod
import flwr as fl
from flwr_datasets import FederatedDataset
save_path = "./my_fl_llama_model"
fds = FederatedDataset(
    dataset=cfg.dataset.name, 
    partitioners={"train": cfg.flower.num_clients}
)

client = fl.client.ClientApp(
    client_fn=gen_client_fn(
        fds,
        tokenizer,
        formatting_prompts_func,
        data_collator,
        cfg.model,
        cfg.train,
        save_path,
    ),
    mods=[fixedclipping_mod]
)

/usr/local/lib/python3.11/dist-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: medalpaca/medical_meadow_medical_flashcards.
  warnings.warn(


In [53]:
from flwr.common import Context

# All docstrings are written by me

def get_on_fit_config():
    """
     • Purpose: This function provides the configuration dictionary sent to each
      client before training begins in a given round.
     • Use Case: Clients can adapt behaviors (e.g., learning rate schedules) based
        on the current round number.
     • Design Pattern: Returns a function (fit_config_fn) that Flower calls at each
      round.

    🧠 Analogy: This is like giving each chef a new cooking instruction every day
    based on how many days the kitchen has been operating.

    """
    def fit_config_fn(server_round: int):
        fit_config = {"current_round": server_round}
        return fit_config

    return fit_config_fn

def fit_weighted_average(metrics):
    """
    • Purpose: Calculates a weighted average of training loss across all clients.
    • Mechanism:
        • Each client’s training loss is scaled by the number of examples it used.
        • This ensures larger datasets have more influence on the final average.
    • Why It Matters: Without weighting, small datasets could skew the overall metric.

    Example:
    # Client 1: 100 examples, loss = 0.5 → 100 * 0.5 = 50
    # Client 2: 200 examples, loss = 0.25 → 200 * 0.25 = 50
    # Weighted average = (50 + 50) / (100 + 200) = 0.333
    """
    # Multiply accuracy of each client by number of examples used
    losses = [num_examples * m["train_loss"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"train_loss": sum(losses) / sum(examples)}

def get_evaluate_fn(model_cfg, save_every_round, total_round, save_path):
    """
     • Purpose: Save the global model periodically during training.
     • Conditions:
       • Skip round 0.
       • Save if it’s the final round or every n rounds (save_every_round).
       • How:
         • Reconstruct the model from the config.
         • Load the current global parameters.
         • Save using the HuggingFace save_pretrained method.

    🧠 Why This Matters: In federated learning, there’s no single centralized
    training process. If something goes wrong, saved checkpoints are your recovery
    point.

    🔁 Return Format: Always returns 0.0, {} because evaluation is optional here
    — the function is used mainly for checkpointing.
    """

    def evaluate(server_round: int, parameters, config):
        # Save model
        if server_round != 0 and (
            server_round == total_round or server_round % save_every_round == 0
        ):
            # Init model
            model = get_model(model_cfg)
            set_parameters(model, parameters)

            model.save_pretrained(f"{save_path}/peft_{server_round}")

        return 0.0, {}

    return evaluate

In [54]:
from flwr.server.strategy import (
    DifferentialPrivacyClientSideFixedClipping
)

def server_fn(context: Context):
    """
    This function returns a configured Flower server app, which will be passed to
    start_simulation() later. It defines:
     • The strategy for aggregation (FedAvg)
     • How rounds behave (e.g., which clients to sample)
     • Optional advanced features like Differential Privacy
    """
    strategy = fl.server.strategy.FedAvg(
        min_available_clients=cfg.flower.num_clients, # total clients
        fraction_fit=cfg.flower.fraction_fit, # ratio of clients to sample
        fraction_evaluate=0.0, # No federated evaluation
        # A (optional) function used to configure a "fit()" round
        on_fit_config_fn=get_on_fit_config(),
        # A (optional) function to aggregate metrics sent by clients
        fit_metrics_aggregation_fn=fit_weighted_average,
        # A (optional) function to execute on the server after each round.
        # In this example the function only saves the global model.
        evaluate_fn=get_evaluate_fn(
            cfg.model,
            cfg.train.save_every_round,
            cfg.flower.num_rounds,
            save_path
        ),
    )

    # Add Differential Privacy to FedAvg strategy
    sampled_clients = cfg.flower.num_clients*strategy.fraction_fit
    strategy = DifferentialPrivacyClientSideFixedClipping(
        strategy,
        noise_multiplier=cfg.flower.dp.noise_mult,
        clipping_norm=cfg.flower.dp.clip_norm,
        num_sampled_clients=sampled_clients
    )

    # Number of rounds to run the simulation
    num_rounds = cfg.flower.num_rounds
    config = fl.server.ServerConfig(num_rounds=num_rounds)

    return fl.server.ServerAppComponents(strategy=strategy, config=config)

server = fl.server.ServerApp(server_fn=server_fn)

In [55]:
from logging import ERROR

client_resources = dict(cfg.flower.client_resources)
backend_setup = {"logging_level": ERROR, "log_to_driver": False}

fl.simulation.run_simulation(
    server_app=server,
    client_app=client,
    num_supernodes=cfg.flower.num_clients,
    backend_config={
        "client_resources": client_resources,
        "init_args": backend_setup
    }
)

INFO :      Starting Flower ServerApp, config: num_rounds=2, no round_timeout
INFO :      
INFO :      [INIT]
INFO :      Requesting initial parameters from one random client
INFO :      Received initial parameters from one random client
INFO :      Starting evaluation of initial global parameters
INFO :      initial parameters (loss, other metrics): 0.0, {}
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 4 clients (out of 20)
INFO :      aggregate_fit: received 4 results and 0 failures
INFO :      aggregate_fit: central DP noise with 0.0025 stdev added
INFO :      fit progress: (1, 0.0, {}, 721.141416849)
INFO :      configure_evaluate: no clients selected, skipping evaluation
INFO :      
INFO :      [ROUND 2]
INFO :      configure_fit: strategy sampled 4 clients (out of 20)
INFO :      aggregate_fit: received 4 results and 0 failures
INFO :      aggregate_fit: central DP noise with 0.0025 stdev added
INFO :      fit progress: (2, 0.0, {}, 1455.48597773

In [57]:
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch
import os

# a. Load the base 4-bit LLM
base = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B-Instruct",
    quantization_config=BitsAndBytesConfig(load_in_4bit=True),
    torch_dtype=torch.bfloat16,
)

# b. Wrap it with your federated fine-tuned adapter
peft_model = PeftModel.from_pretrained(
    base,
    os.path.join(save_path, "peft_2"),    # path to your adapter folder
)

# c. Merge the base model with the adapter
merged_model = peft_model.merge_and_unload()

# d. Push the merged model to HuggingFace Hub
merged_model.push_to_hub("tuhanasinan/llama32_1b_federated_finetuned_250steps")

/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            

CommitInfo(commit_url='https://huggingface.co/tuhanasinan/llama32_1b_federated_finetuned_250steps/commit/c99a5874c8c69e0ebfcdb3a654248c8b6b2f5abe', commit_message='Upload LlamaForCausalLM', commit_description='', oid='c99a5874c8c69e0ebfcdb3a654248c8b6b2f5abe', pr_url=None, repo_url=RepoUrl('https://huggingface.co/tuhanasinan/llama32_1b_federated_finetuned_250steps', endpoint='https://huggingface.co', repo_type='model', repo_id='tuhanasinan/llama32_1b_federated_finetuned_250steps'), pr_revision=None, pr_num=None)

In [59]:
!pip install llama_index

INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 91.0 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.3/303.3 kB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [61]:
from llama_index.llms.huggingface import HuggingFaceLLM
from IPython.display import display, Markdown


llm = HuggingFaceLLM(
    model_name="tuhanasinan/llama32_1b_federated_finetuned_250steps",
    tokenizer_name="meta-llama/Llama-3.2-1B-Instruct" 
)
prompt = "What are the management strategies for antiphospholipid syndrome?"
response = llm.complete(prompt)

display(Markdown(str(response)))

ModuleNotFoundError: No module named 'llama_index.llms.huggingface'

In [62]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from IPython.display import display, Markdown

# Model ve tokenizer adlarını belirle
model_name = "tuhanasinan/llama32_1b_federated_finetuned_250steps"
tokenizer_name = "meta-llama/Llama-3.2-1B-Instruct" 

# Tokenizer'ı yükle
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# Modeli yükle (GPU varsa otomatik olarak GPU'ya yükler)
# torch_dtype=torch.bfloat16 performansı artırabilir
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    torch_dtype=torch.bfloat16, 
    device_map="auto" # Modeli uygun cihazlara (GPU/CPU) dağıtır
)

# Metin üretimi için bir pipeline oluştur
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

# Prompt'u tanımla ve metni üret
prompt = "What are the management strategies for antiphospholipid syndrome?"
# Pipeline'a ek parametreler verebilirsin (max_new_tokens vb.)
sequences = text_generator(
    prompt,
    max_new_tokens=250,  # Üretilecek maksimum yeni token sayısı
    do_sample=True,      # Örnekleme yaparak daha çeşitli sonuçlar al
    temperature=0.7,     # Daha yaratıcı (yüksek) veya deterministik (düşük) sonuçlar için
    top_k=50,            # En olası K token arasından seçim yap
    top_p=0.95,          # Kümülatif olasılığı P olan tokenlar arasından seçim yap
    num_return_sequences=1, # Kaç tane sonuç döndürüleceği
)

# Sonucu göster
# Pipeline genellikle [{'generated_text': '...'}] formatında bir liste döndürür
if sequences and len(sequences) > 0:
    response_text = sequences[0]['generated_text']
    display(Markdown(response_text))
else:
    print("Model yanıt üretemedi.")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


What are the management strategies for antiphospholipid syndrome? 
Management strategies for antiphospholipid syndrome (APS) include:
1. Anticoagulation therapy: The most common treatment for APS is anticoagulation therapy with specific anticoagulant medications, such as low molecular weight heparin or fondaparick.
2. Low molecular weight heparin therapy: Low molecular weight heparin is often used as a first-line treatment for APS.
3. Fibrinolytic therapy: Fibrinolytic therapy, such as recombinant activated protein C (rAPC) or activated protein C (APC), may be used in some cases of APS.
4. Corticosteroid therapy: Corticosteroid therapy may be used to treat APS, especially in cases where anticoagulation therapy is not effective.
5. Vasculature-based therapy: Vasculature-based therapy, such as carotid endarterectomy or carotid artery bypass grafting, may be used to address arterial complications associated with APS.
6. Fetal-maternal transfusion: Fetal-maternal transfusion may be used to prevent complications associated with APS, such as placental abruption.
7. Lifestyle modifications: Lifestyle modifications, such as